<a href="https://colab.research.google.com/github/prithvikavoori/PySparkMllib/blob/main/DogFoodChemicalCLassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

we are going to build a RandomForest classification model for dog food company to predict which chemicals in the data are causing the food to stale quickly. this model would help us to identify which chemical is causing the damage to the food. the data is available in git hub repository. since this model is being build in google colaboratory always add data to the files after reconnecting 

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [4]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [5]:
# install findspark using pip
!pip install -q findspark

In [6]:
import findspark
findspark.init()

In [7]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [8]:
# Load training data
data = spark.read.csv('/content/dog_food.csv',inferSchema=True,header=True)

In [9]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [10]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [11]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [12]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [14]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [15]:
output = assembler.transform(data)

In [16]:
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier

In [17]:
dtc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [18]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
final_data = output.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [20]:
rfc_model = rfc.fit(final_data)
dtc_model = dtc.fit(final_data)

In [21]:
rfc_model.featureImportances

SparseVector(4, {0: 0.021, 1: 0.0187, 2: 0.9343, 3: 0.0259})

In [26]:
dtc_model.featureImportances

SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

from the feature importance of the 2 models we can conclude that chemical at index 2 which is chemical c is the one spoiling the dog food 